# Canonical Tests

The goal of the work we performed on behalf of OGC/USGS was to enable users on QHub (and perhaps plain JupyterHub) to:
- [ ] run long-running notebooks or scripts
- [ ] run notebooks and scripts as cronjobs

And as a stretch goal:
- [ ] run complex workflows that require multiple steps / noteboks or scripts

Additional requirements:
- the notebook or script should work even after the user's JupyterLab session ends
- the notebook or script can connect to the Dask-Gateway and launch a Dask cluster

This notebooks will serve as a "unit test" for the above features, for more details [see this issue](https://github.com/Quansight/ogc-management/issues/6).

## `kbatch`

The first two features outline above will be handled by [`kbatch`](https://github.com/kbatch-dev/kbatch). `kbatch` consists of two major components, the frontend `kbatch` and backend `kbatch-proxy`. The user submits job requests to `kbatch-proxy` and `kbatch-proxy` submits those job requests to the Kubernetes API. 

> NOTE:
> At the present, no additional features have been added to `kbatch`, we have simply integrated `kbatch-proxy` into QHub. A feature enhancement PR will need to be opened on the `kbatch` repo in order to enable cronjobs.

### Setup

In order to use `kbatch` in it's current form, some basic setup is required of the user. Going forward, we will assume that `kbatch-proxy` has been correctly integrated into QHub.

1. Create or modify a conda environment by adding `kbatch`. And activate this conda environment.

```
pip install kbatch
```

2. Create a JupyterHub API token and configure `kbatch` to talk to the `kbatch-proxy` service.

```
kbatch configure --kbatch-url http://kbatch-kbatch-proxy.dev.svc.cluster.local --token <JUPYTERHUB_API_TOKEN>
```

3. Submit a job to `kbatch`

```
kbatch job submit --name=list-files --image=alpine --command='["ls", "-lh"]'
```

### Run this notebook

To run this notebook as a job, you will need an image with `papermill` (or a similar CLI tool). 

Create a configuration file, `kbatch_nb_job.yaml` like the one below:
```yaml
# filename: kbatch_nb_job.yaml
name: nb-job
image: mcr.microsoft.com/planetary-computer/python:latest
args:
  - papermill
  - kbatch_nb.ipynb
code: kbatch_nb.ipynb
```

Then run:
```
kbatch job submit -f kbatch_nb_job.yaml
```

In [9]:
import time

In [11]:
with open('kbatch_nb_output.txt', 'w') as f:
    for i in range(0,10):
        current_time = time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime())
        time.sleep(1)
        f.write(f'{current_time}: {i}\n')